In [ ]:
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from utils import *
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Specify the data folder paths where all datasets as stored
config = load_config()
incoming_data_path = config['paths']['incoming_data']
processed_data_path = config['paths']['data']

nbs_input_data_path = os.path.join(incoming_data_path,"nbs")
# The projection system for Jamaica
epsg_jamaica = 3448

In [ ]:
# Specify the input layers we want to create buffers around and intersect
natural_layers = ["nsmdb-mangroves.gpkg",
                 "nsmdb-seagrass.gpkg",
                 "nsmdb-reefs_jan06_NEPA.gpkg"]
output_layer_name = ["mangroves",
                     "seagrass",
                     "corals"]
# Buffer distances in meters
buffer_distances = [100,250,500,1000]

In [ ]:
# Create a for loop over the buffer distances and the layer 
# Add the buffer to the geometry of the layer 
# Write the result to a new geopackage file
for buffer in buffer_distances:
    for i,(input_layer,output_layer) in enumerate(list(zip(natural_layers,output_layer_name))):
        # Read a Geopackage layer or shapefile
        get_layer = gpd.read_file(os.path.join(nbs_input_data_path,
                                                input_layer))
        # Reproject to Jamaica projection system
        get_layer = get_layer.to_crs(epsg=epsg_jamaica)
        # Create buffer around geometry 
        get_layer["geometry"] = get_layer.progress_apply(
                        lambda x: x.geometry.buffer(buffer),
                        axis=1)
        get_layer.to_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{output_layer}_buffer.gpkg"),
                                          layer=f"{buffer}m",
                                         driver="GPKG")

In [ ]:
# Intersect the different buffer layers to find common areas
# Buffer distances in meters
buffer_distances = [250,500,1000]
intersection_layers = ["mangroves",
                     "seagrass",
                     "corals"]
for buffer in buffer_distances:
    for i in range(len(intersection_layers)-1):
        name_i = intersection_layers[i] 
        for j in intersection_layers[i+1:]:
            name_j = j
            layer_i = gpd.read_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{name_i}_buffer.gpkg"),
                                          layer=f"{buffer}m")
            layer_i.rename(columns={"ID":f"{name_i}_id"},inplace=True)
            layer_j = gpd.read_file(os.path.join(processed_data_path,
                                       "nbs",
                                       f"{name_j}_buffer.gpkg"),
                                          layer=f"{buffer}m")
            layer_j.rename(columns={"ID":f"{name_j}_id"},inplace=True)
            common_ij = gpd.sjoin(layer_i,layer_j,
                                  how="inner",
                                  predicate="intersects").reset_index()
            common_ij.rename(columns={"geometry":"layer_i_geometry"},
                             inplace=True)
            common_ij = pd.merge(common_ij,
                                 layer_j[[f"{name_j}_id","geometry"]],
                                 how="left",on=[f"{name_j}_id"])
            common_ij.rename(columns={"geometry":"layer_j_geometry"},
                             inplace=True)
            common_ij["geometry"] = common_ij.progress_apply(
                lambda x:x.layer_i_geometry.intersection(x.layer_j_geometry)
                ,axis=1)
            common_ij.drop(["layer_i_geometry","layer_j_geometry"],
                           axis=1,
                           inplace=True)
            common_ij = gpd.GeoDataFrame(common_ij,
                             geometry="geometry",
                             crs=f"EPSG:{epsg_jamaica}")
            common_ij.to_file(os.path.join(processed_data_path,
                            "nbs",
                            f"{name_i}_{name_j}_intersections.gpkg"),
                            layer=f"{buffer}m",
                            driver="GPKG")